In [ ]:
from google.colab import userdata
vantage_api_key = userdata.get('vantage_api_key')

In [ ]:
symbol = "KLAC"
# note that Formatted String Literal is used here
url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={vantage_api_key}"

In [ ]:
!pip install requests

In [ ]:
import requests
r = requests.get(url)

In [ ]:
from google.colab import userdata

user_id = userdata.get('snowflake_userid')
password = userdata.get('snowflake_password')
account = userdata.get('snowflake_account')
database = userdata.get('snowflake_database')
vwh = userdata.get('snowflake_vwh')

In [ ]:
!pip install snowflake-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.4/479.4 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 2.0.0
    Uninstalling cffi-2.0.0:
      Successfully uninstalled cffi-2.0.0


In [ ]:
import snowflake.connector

def return_snowflake_conn():
    # Establish a connection to Snowflake
    conn = snowflake.connector.connect(
        user=user_id,
        password=password,
        account=account,
        warehouse=vwh,
        database=database,
        schema="raw",
        role="TRAINING_ROLE"
    )
    # Create a cursor object
    return conn.cursor()

In [ ]:
def extract_last_90d_price(symbol):
    vantage_api_key = userdata.get('vantage_api_key')
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    results = []

    # Get the daily data (dict keyed by date strings)
    daily_data = data.get("Time Series (Daily)", {})

    # Sort the dates in descending order (newest first)
    sorted_dates = sorted(daily_data.keys(), reverse=True)

    # Take only the latest 90 days
    for d in sorted_dates[:90]:
        stock_info = daily_data[d].copy()
        stock_info["date"] = d
        results.append(stock_info)

    return results


In [ ]:
data = extract_last_90d_price(symbol)

In [ ]:
for d in data:
  print(d)

{'1. open': '1056.5400', '2. high': '1068.6600', '3. low': '1046.1601', '4. close': '1064.2900', '5. volume': '505437', 'date': '2025-09-26'}
{'1. open': '1058.6100', '2. high': '1066.9950', '3. low': '1045.8043', '4. close': '1059.1000', '5. volume': '710581', 'date': '2025-09-25'}
{'1. open': '1066.3200', '2. high': '1076.0000', '3. low': '1057.6400', '4. close': '1068.6700', '5. volume': '868147', 'date': '2025-09-24'}
{'1. open': '1067.8500', '2. high': '1075.5200', '3. low': '1060.7900', '4. close': '1071.2000', '5. volume': '829643', 'date': '2025-09-23'}
{'1. open': '1038.8900', '2. high': '1072.9700', '3. low': '1037.5000', '4. close': '1071.2500', '5. volume': '1020716', 'date': '2025-09-22'}
{'1. open': '1051.9000', '2. high': '1052.4800', '3. low': '1029.3150', '4. close': '1044.8100', '5. volume': '1586978', 'date': '2025-09-19'}
{'1. open': '1030.8300', '2. high': '1076.7900', '3. low': '1025.7850', '4. close': '1046.6900', '5. volume': '1825038', 'date': '2025-09-18'}
{'1

In [ ]:
import pandas as pd

def transform(data):

    df = pd.DataFrame(data)

    # Rename columns
    df = df.rename(columns={
        "1. open": "open",
        "2. high": "high",
        "3. low": "low",
        "4. close": "close",
        "5. volume": "volume"
    })

    # Convert types
    df["date"] = pd.to_datetime(df["date"])
    df[["open", "high", "low", "close"]] = df[["open", "high", "low", "close"]].astype(float)
    df["volume"] = df["volume"].astype(int)

    # Sort by date (most recent first, optional)
    df = df.sort_values("date", ascending=False).reset_index(drop=True)

    return df



In [ ]:
records = transform(data)

In [ ]:
len(records)

90

In [ ]:
records

,open,high,low,close,volume,date
0,1056.54,1068.660,1046.1601,1064.29,505437,2025-09-26
1,1058.61,1066.995,1045.8043,1059.10,710581,2025-09-25
2,1066.32,1076.000,1057.6400,1068.67,868147,2025-09-24
3,1067.85,1075.520,1060.7900,1071.20,829643,2025-09-23
4,1038.89,1072.970,1037.5000,1071.25,1020716,2025-09-22
...,...,...,...,...,...,...
85,774.71,792.180,768.5983,789.06,1292643,2025-05-27
86,747.75,761.100,744.8700,757.17,941948,2025-05-23
87,766.46,781.760,762.0001,767.51,682541,2025-05-22
88,778.32,799.720,775.1800,778.94,1236816,2025-05-21


In [ ]:
def load(con, records):
#CREATING DB, SCHEMA AND TABLE FOR LOADING
    # Step 1: Select your database
    con.execute(f"USE DATABASE {database}")

    # Step 2: Create the schema if it doesn’t exist
    con.execute("CREATE SCHEMA IF NOT EXISTS raw")

    # Step 3: Create the table inside the 'raw' schema
    create_table_query = """
    CREATE TABLE IF NOT EXISTS raw.stock_data (
        symbol STRING NOT NULL,
        date DATE NOT NULL,
        open FLOAT,
        high FLOAT,
        low FLOAT,
        close FLOAT,
        volume FLOAT,
        PRIMARY KEY (symbol, date)
    )
    """
    con.execute(create_table_query)

    print("Table 'raw.stock_data' created or already exists.")

    # Show tables in the 'raw' schema
    con.execute("SHOW TABLES IN raw")
#LOADING INTO TABLE
    tables = con.fetchall()

    for table in tables:
      print(table)

    delete_query = "DELETE FROM raw.stock_data WHERE symbol = %s"
    con.execute(delete_query, (symbol,))

    try:
        con.execute("BEGIN")  # Start transaction

        insert_query = """
            INSERT INTO raw.stock_data (symbol, date, open, high, low, close, volume)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """

        # Loop through DataFrame and insert data
        for _, row in records.iterrows():
            con.execute(insert_query, (
                symbol,
                row["date"].strftime('%Y-%m-%d'),  # Convert date to string
                row["open"],
                row["high"],
                row["low"],
                row["close"],
                row["volume"]
            ))

        con.execute("COMMIT")  # Commit transaction

        print("Data inserted successfully!")

    except Exception as e:
        con.execute("ROLLBACK")  # Rollback in case of an error
        print("Error occurred:", e)

In [ ]:
con = return_snowflake_conn()
load(con, records)


Table 'raw.stock_data' created or already exists.
(datetime.datetime(2025, 9, 19, 0, 53, 22, 404000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), 'MARKET_DATA', 'USER_DB_JACKAL', 'RAW', 'TABLE', '', '', 130, 7680, 'TRAINING_ROLE', '1', 'OFF', 'OFF', 'OFF', None, None, 'N', 'N', 'ROLE', 'N', 'N', 'N', 'N', 'N')
(datetime.datetime(2025, 9, 19, 1, 9, 24, 275000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), 'MY_FORECASTS_2025_09_19', 'USER_DB_JACKAL', 'RAW', 'TABLE', '', '', 14, 2560, 'TRAINING_ROLE', '1', 'OFF', 'OFF', 'OFF', None, None, 'N', 'N', 'ROLE', 'N', 'N', 'N', 'N', 'N')
(datetime.datetime(2025, 9, 12, 0, 48, 6, 657000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), 'PROD_NEW_DATA', 'USER_DB_JACKAL', 'RAW', 'TABLE', '', '', 4620, 38912, 'TRAINING_ROLE', '1', 'OFF', 'OFF', 'OFF', None, None, 'N', 'N', 'ROLE', 'N', 'N', 'N', 'N', 'N')
(datetime.datetime(2025, 9, 12, 0, 48, 46, 584000, tzinfo=<DstTzInfo 'America/Los_Ang

In [ ]:
def check_table_stats(table):
    result = con.execute(f"SELECT * FROM {table}")
    df = con.fetch_pandas_all()
    print(df.head())
    print(len(df))
    print(" Record_length : ", len(records))

In [ ]:
data = extract_last_90d_price(url)
records = transform(data)
con = return_snowflake_conn()
load(con, records)
target_table = f"{database}.raw.stock_data"

check_table_stats(target_table)


Table 'raw.stock_data' created or already exists.
(datetime.datetime(2025, 9, 19, 0, 53, 22, 404000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), 'MARKET_DATA', 'USER_DB_JACKAL', 'RAW', 'TABLE', '', '', 130, 7680, 'TRAINING_ROLE', '1', 'OFF', 'OFF', 'OFF', None, None, 'N', 'N', 'ROLE', 'N', 'N', 'N', 'N', 'N')
(datetime.datetime(2025, 9, 19, 1, 9, 24, 275000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), 'MY_FORECASTS_2025_09_19', 'USER_DB_JACKAL', 'RAW', 'TABLE', '', '', 14, 2560, 'TRAINING_ROLE', '1', 'OFF', 'OFF', 'OFF', None, None, 'N', 'N', 'ROLE', 'N', 'N', 'N', 'N', 'N')
(datetime.datetime(2025, 9, 12, 0, 48, 6, 657000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), 'PROD_NEW_DATA', 'USER_DB_JACKAL', 'RAW', 'TABLE', '', '', 4620, 38912, 'TRAINING_ROLE', '1', 'OFF', 'OFF', 'OFF', None, None, 'N', 'N', 'ROLE', 'N', 'N', 'N', 'N', 'N')
(datetime.datetime(2025, 9, 12, 0, 48, 46, 584000, tzinfo=<DstTzInfo 'America/Los_Ang